In [12]:
import serial
import re
import time
import pygame
import sys
import numpy as np
import math

In [13]:
class Position:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.z = 0

class Orientation:
    def __init__(self):
        self.roll = 0
        self.pitch = 0

class Pose:
    def __init__(self):
        self.position = Position()
        self.orientation = Orientation()

    def update_pose(self, pose_array):
        self.position.x = pose_array[0]
        self.position.y = pose_array[1]
        self.position.z = pose_array[2]
        self.orientation.pitch = pose_array[3]
        self.orientation.roll = pose_array[4]

    def __str__(self):
        return f"X: {self.position.x}, Y: {self.position.y}, Z: {self.position.z}, P: {self.orientation.pitch}, R: {self.orientation.roll}"

In [14]:
import serial
import re
import time

from Pose import Pose

class Robot:
    def __init__(self, port):
        self.ser = serial.Serial(port)
        # self.ser.write(b"HOME \r")
        # time.sleep(160)
        self.ser.write(b"CON \r")
        time.sleep(5)
        self.ser.write(b"~\r")
        check = ''
        while True:
            check += self.ser.read_all().decode('ascii')
            print(check)
            if bool(re.search("EXIT", check)):
                self.ser.write(b"~\r")
                break
            if bool(re.search("MANUAL MODE!", check)):
                break
            time.sleep(0.1)
        response = self.ser.read_all()
        print(response.decode('ascii'))
        '''
          pattern = "X:([\s-]\d+)\s*Y:([\s-]\d+)\s*Z:([\s-]\d+)\s*P:([\s-]\d+)\s*R:([\s-]\d+)"
        # X: 3423, Y: -353, Z: 3423, P: -201, R: 76
        match = re.search(pattern, response.decode('ascii'))
        '''
        self.ser.write(b"J\r")
        self.mode = 'J'
        self.manual_mode = True
        time.sleep(5)

    def close(self):
        self.ser.close()
    
    '''
    Starts from the initial position and updates the position
    '''

    def move_robot_z(self, increment):
        print("z change")
        if self.mode == 'J':
            self.ser.write(b"X\r")
            time.sleep(0.1)
            self.mode = 'X'
        if increment > 0:
            self.ser.write(b"3\r")
            time.sleep(0.1)
        else:
            self.ser.write(b"E\r")
            time.sleep(0.1)

    def move_robot_x(self, increment):
        if self.mode == 'J':
            self.ser.write(b"X\r")
            time.sleep(0.1)
            self.mode = 'X'
        if increment > 0:
            self.ser.write(b"1\r")
            time.sleep(0.1)
        else:
            self.ser.write(b"Q\r")
            time.sleep(0.1)

    def move_robot_y(self, increment):
        if self.mode == 'J':
            self.ser.write(b"X\r")
            time.sleep(0.1)
            self.mode = 'X'
        if increment > 0:
            self.ser.write(b"2\r")
            time.sleep(0.1)
        else:
            self.ser.write(b"W\r")
            time.sleep(0.1)

    def move_robot_pitch(self, increment):
        if self.mode == 'X':
            self.ser.write(b"J\r")
            time.sleep(0.1)
            self.mode = 'J'
        if increment > 0:
            self.ser.write(b"4\r")
            time.sleep(0.1)
        else:
            self.ser.write(b"R\r")
            time.sleep(0.1)

    def move_robot_roll(self, increment):
        if self.mode == 'X':
            self.ser.write(b"J\r")
            time.sleep(0.1)
            self.mode = 'J'
        if increment > 0:
            self.ser.write(b"5\r")
            time.sleep(0.1)
        else:
            self.ser.write(b"T\r")
            time.sleep(0.1)

    def check_messages(self):
        response = self.ser.read_all().decode('ascii')
        if bool(re.search("DISABLED", response)):
            self.ser.write(b"CON \r")
        print(response)

    def print_pose(self):
        print(self.pose)

In [15]:
robot = Robot("COM5")

CONTROL ENABLED.                   
*** IMPACT PROTECTION axis 9
>
CONTROL ENABLED.                   
*** IMPACT PROTECTION axis 9
>
EXIT MANUAL MODE...
>
>



In [16]:
# robot.ser.write(b"HOME \r")
# time.sleep(160)

In [17]:
def process_input(axes, buttons):
    threshold = 0.9
    # Camera positions

    # if buttons[0] == 1:
    #     print("camera position 1")
    # elif buttons[1] == 1:
    #     print("camera position 2")
    # elif buttons[2] == 1:
    #     print("camera position 3")
    # elif buttons[3] == 1:
    #     print("camera position 4")

    # change z axes


    if buttons[4] == 1 and buttons[6] == 0:
        robot.move_robot_z(-20)
        print("decrease z by scale -> ")
    elif buttons[4] == 0 and buttons[6] == 1:
        robot.move_robot_z(20)
        print("increase z by scale -> " )
    # change bisturi x and y

    if axes[0] >= threshold:
        robot.move_robot_y(-20)
        print("decrease y by scale -> " + str(axes[0]))
    elif axes[0] <= -threshold:
        robot.move_robot_y(20)
        print("decrease y by scale -> " + str(axes[0]))

    if axes[1] >= threshold:
        robot.move_robot_x(-20)
        print("decrease x by scale -> " + str(axes[1]))
    elif axes[1] <= -threshold:
        robot.move_robot_x(20)
        print("decrease x by scale -> " + str(axes[1]))

    # change the pitch and roll

    if axes[2] >= threshold:
        robot.move_robot_roll(-20)
        print("decrease roll by scale -> " + str(axes[2]))
    elif axes[2] <= -threshold:
        robot.move_robot_roll(20)
        print("increase roll by scale -> " + str(-axes[2]))

    if axes[3] >= threshold:
        robot.move_robot_pitch(-20)
        print("decrease pitch by scale -> " + str(axes[3]))
    elif axes[3] <= -threshold:
        robot.move_robot_pitch(20)
        print("increase pitch x by scale -> " + str(-axes[3]))

In [18]:
pygame.init()
pygame.joystick.init()

joystick = pygame.joystick.Joystick(0)
joystick.init()

running = True

r1_pressed = False
l1_pressed = False

while running:
    start_time = time.time()
        
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            robot.close()
            pygame.quit()

    axes = [round(joystick.get_axis(i), 3) for i in range(joystick.get_numaxes())]
    buttons = [joystick.get_button(i) for i in range(joystick.get_numbuttons())]
    # print(axes)
    # print(buttons)
    process_input(axes, buttons)
    time.sleep(0.1)
    robot.check_messages()
    # end_time = time.time()
    # if (r1_pressed) or (l1_pressed):
    #     print(end_time - start_time)


MANUAL MODE!
>
 JOINT MODE.              


























decrease x by scale -> -1.0
 XYZ MODE.                
decrease x by scale -> -1.0

decrease x by scale -> -1.0

decrease x by scale -> -1.0

decrease x by scale -> -1.0

decrease x by scale -> -1.0

decrease x by scale -> -1.0

decrease x by scale -> -1.0

decrease x by scale -> -1.0


decrease y by scale -> -1.0

decrease y by scale -> -1.0

decrease y by scale -> -1.0























CONTROL ENABLED.
>


















decrease x by scale -> -1.0

decrease x by scale -> -1.0




















decrease x by scale -> 1.0








































































































decrease x by scale -> -1.0

decrease x by scale -> -1.0















z change
increase z by scale -> 

z change
increase z by scale -> 

z change
increase z by scale -> 





z change
decrease z by scale -> 

z change
decrease z by scale -> 




z change
decrease z by scale -> 




z c

KeyboardInterrupt: 

In [ ]:
robot.close()

In [ ]:
robot.move_robot_z(-100)

SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))